In [1]:
from datasets import load_dataset
import pandas as pd
import pickle
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

train = load_dataset('lewtun/drug-reviews', split='train')
test = load_dataset('lewtun/drug-reviews', split='test')
df_train = pd.DataFrame(train)
df_test = pd.DataFrame(test)

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [8]:
df_space = pd.read_csv('./data/entity_datasets/world_place.csv')
df_space.head()

,name,entity_subtype,country,latitude,longitude,has_thumbnail,page_views,entity_type,total_area,population,is_test
0,Tancredo Neves Pantheon of the Fatherland and ...,Monument,Brazil,-15.80170,-47.86030,1,7521.0,structure,NaN,NaN,False
1,Sembuwatta Lake,AmusementParkAttraction,Sri_Lanka,7.43694,80.69970,1,16340.0,structure,NaN,NaN,False
2,Ford Rotunda,AmusementParkAttraction,United_States,42.31210,-83.17670,1,40773.0,structure,NaN,NaN,False
3,Afsluitdijk,Bridge,Netherlands,53.00000,5.16667,1,227321.0,structure,NaN,NaN,False
4,Mudeirej Bridge,Bridge,Lebanon,33.80080,35.72710,1,5623.0,structure,NaN,NaN,True


In [9]:
df_train = df_train.drop(['Unnamed: 0'], axis=1)
df_test = df_test.drop(['Unnamed: 0'], axis=1)
df_train['is_test'] = False
df_test['is_test'] = True

In [15]:
top_30_classes = df_test['condition'].value_counts().nlargest(30).index.tolist()
df_train_filtered = df_train[df_train['condition'].isin(top_30_classes)]
df_test_filtered = df_test[df_test['condition'].isin(top_30_classes)]
df_train_filtered['cat'] = df_train_filtered['condition'].astype('category').cat.codes
df_test_filtered['cat'] = df_test_filtered['condition'].astype('category').cat.codes
mapping_dict = dict(enumerate(df_test_filtered['condition'].astype('category').cat.categories))
mapping_dict_ = dict(enumerate(df_train_filtered['condition'].astype('category').cat.categories))

print(mapping_dict == mapping_dict_)
len(df_train_filtered), len(df_test_filtered)
df_train_filtered.head()

True


/Users/tianyuhan/.local/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/Users/tianyuhan/.local/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


(107943, 36124)

,drugName,condition,review,rating,date,usefulCount,is_test,cat
1,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",8.0,"April 27, 2010",192,False,0
2,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5.0,"December 14, 2009",17,False,6
3,Ortho Evra,Birth Control,"""This is my first time using any form of birth...",8.0,"November 3, 2015",10,False,6
4,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",9.0,"November 27, 2016",37,False,21
6,Levonorgestrel,Emergency Contraception,"""He pulled out, but he cummed a bit in me. I t...",1.0,"March 7, 2017",5,False,12


In [16]:
df = pd.concat([df_train_filtered, df_test_filtered])
len(df)

144067

In [17]:
df.to_csv('./data/entity_datasets/drug.csv', index=False)

In [18]:
top_30_classes

['Birth Control',
 'Depression',
 'Pain',
 'Anxiety',
 'Acne',
 'Bipolar Disorde',
 'Weight Loss',
 'Insomnia',
 'Obesity',
 'ADHD',
 'Emergency Contraception',
 'Vaginal Yeast Infection',
 'Diabetes, Type 2',
 'High Blood Pressure',
 'Smoking Cessation',
 'Abnormal Uterine Bleeding',
 'Bowel Preparation',
 'Migraine',
 'ibromyalgia',
 'Anxiety and Stress',
 'Constipation',
 'Major Depressive Disorde',
 'Chronic Pain',
 'Panic Disorde',
 'Migraine Prevention',
 'Urinary Tract Infection',
 'Opiate Dependence',
 'Osteoarthritis',
 'Muscle Spasm',
 'Erectile Dysfunction']

In [9]:
path = '/home/than/DeepLearning/world-models/results/tuned/Llama-2-70b-hf/drug/condition/probes/lsq_train.none.none.last.condition.p'
probe_results = pickle.load(open(path, 'rb'))
df = probe_results['projections']
df[70].head()

,projection,is_test
0,0,False
1,6,False
2,6,False
3,21,False
4,6,False
